In [1]:
import pandas as pd
import zipfile
import io
import numpy as np

from keras.preprocessing.text import text_to_word_sequence, Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers import Embedding
from keras.layers import Bidirectional
from keras.layers import GRU
from keras.layers import Dropout
from keras.layers import BatchNormalization

from keras.callbacks import ModelCheckpoint, TensorBoard


from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder, LabelEncoder

Using TensorFlow backend.


In [2]:
def load_dataset(filename):
  df = pd.read_excel(filename)
  intent = df["intent"]
  unique_intent = list(set(intent))
  text = list(df["text"])
  return (intent, unique_intent, text, df)


def load_word_vectors(filepath,vocab):
    with zipfile.ZipFile(filepath) as zfile:
        for finfo in zfile.infolist():
            ifile = zfile.open(finfo)
            textStream = io.TextIOWrapper(ifile, encoding='utf-8')
            #n, d = map(int, textStream.readline().split())
            #print("Number of tokens = "+str(n))
            data = {}
            for line in textStream:
                tokens = line.rstrip().split(' ')
                word=tokens[0]
                if(word in vocab):
                    data[word] = np.asarray(tokens[1:], dtype='float32')
            return data

        
def build_embedding_matrix(tok,vectors,embedding_size):
    vocab_size=len(tok.word_index) + 1
    embedding_matrix = np.zeros((vocab_size, embedding_size))
    for word, i in tok.word_index.items():
        embedding_vector = vectors.get(word)
        if embedding_vector is not None:
            embedding_matrix[i] = embedding_vector
    return embedding_matrix


def get_vocab(sentences):
    text_words=list(map(lambda x: text_to_word_sequence(x,lower=False),text))
    vocab=set()
    for words in text_words:
        vocab.update(words)
    return vocab


def prepare_training_data(text,intent,max_length,embedding_size):
    tok = Tokenizer(lower=False)
    tok.fit_on_texts(text)
    encoded_docs = tok.texts_to_sequences(text)
    padded_docs = pad_sequences(encoded_docs, maxlen=max_length, padding='post')
    embedding_matrix = build_embedding_matrix(tok,glove_vectors,embedding_size)
    vocab_size = len(tok.word_index) + 1
    le = LabelEncoder()
    labels_encoded=le.fit_transform(intent)
    ohe = OneHotEncoder(sparse=False,categories='auto')
    output_one_hot = ohe.fit_transform(labels_encoded.reshape(-1, 1))
    train_X, val_X, train_Y, val_Y = train_test_split(padded_docs, output_one_hot, shuffle = True, test_size = 0.2)
    return (train_X, val_X, train_Y, val_Y, vocab_size, embedding_matrix)


def build_model(embedding_matrix,max_length, use_pre_trained_vectors=True):
    model = Sequential()
    if use_pre_trained_vectors:
        e = Embedding(vocab_size, embedding_size, weights=[embedding_matrix], 
                      input_length=max_length, trainable=False)
    else:
        e = Embedding(vocab_size, embedding_size, input_length=max_length)
    
    model.add(e)
    model.add(Bidirectional(GRU(128)))
    model.add(Dense(64, activation = "relu"))
    model.add(Dropout(0.5))
    model.add(Dense(64, activation = "relu"))
    model.add(Dropout(0.5))
    model.add(BatchNormalization())
    model.add(Dense(21, activation = "softmax"))
    model.compile(loss = "categorical_crossentropy", optimizer = "adam", metrics = ["accuracy"])
    return model

In [3]:
intent, unique_intent, text, df = load_dataset("../data/chatbot_questions.xlsx")

In [4]:
vocab=get_vocab(text)

In [5]:
glove_vectors=load_word_vectors("../data/pre-trained-vectors/crawl-300d-2M.vec.zip",vocab)

In [6]:
max_length=20
embedding_size = 300

train_X, val_X, train_Y, val_Y, vocab_size, embedding_matrix = prepare_training_data(text,intent,max_length,embedding_size)

In [15]:
model = build_model(embedding_matrix,max_length,use_pre_trained_vectors=True)

In [16]:

#model.summary()

In [17]:
filepath="../data/model/pre-trained_vectors/model-epoch-{epoch:02d}-val_acc-{val_acc:.2f}.hdf5"
log_path="../logs/pre-trained/"
tboard_callback=TensorBoard(log_dir=log_path)
checkpoint = ModelCheckpoint(filepath, monitor='val_acc', verbose=1, save_best_only=True, mode='max')
callbacks_list = [checkpoint,tboard_callback]

In [18]:
model.fit(train_X, train_Y, epochs = 100, batch_size = 32, validation_data = (val_X, val_Y),callbacks=callbacks_list)

Train on 890 samples, validate on 223 samples
Epoch 1/100
890/890 [==============================] - 2s 2ms/step - loss: 3.2468 - acc: 0.1000 - val_loss: 2.6804 - val_acc: 0.2646

Epoch 00001: val_acc improved from -inf to 0.26457, saving model to ../data/model/pre-trained_vectors/model-epoch-01-val_acc-0.26.hdf5
Epoch 2/100
890/890 [==============================] - 1s 1ms/step - loss: 2.8702 - acc: 0.1528 - val_loss: 2.3337 - val_acc: 0.4798

Epoch 00002: val_acc improved from 0.26457 to 0.47982, saving model to ../data/model/pre-trained_vectors/model-epoch-02-val_acc-0.48.hdf5
Epoch 3/100
890/890 [==============================] - 1s 1ms/step - loss: 2.5374 - acc: 0.2708 - val_loss: 1.9596 - val_acc: 0.5605

Epoch 00003: val_acc improved from 0.47982 to 0.56054, saving model to ../data/model/pre-trained_vectors/model-epoch-03-val_acc-0.56.hdf5
Epoch 4/100
890/890 [==============================] - 1s 1ms/step - loss: 2.2658 - acc: 0.3607 - val_loss: 1.7334 - val_acc: 0.6457

Epoch 0

890/890 [==============================] - 1s 1ms/step - loss: 0.3810 - acc: 0.8955 - val_loss: 0.3210 - val_acc: 0.9103

Epoch 00037: val_acc did not improve from 0.92377
Epoch 38/100
890/890 [==============================] - 1s 1ms/step - loss: 0.3839 - acc: 0.9000 - val_loss: 0.3201 - val_acc: 0.9148

Epoch 00038: val_acc did not improve from 0.92377
Epoch 39/100
890/890 [==============================] - 1s 1ms/step - loss: 0.3927 - acc: 0.8966 - val_loss: 0.3183 - val_acc: 0.9103

Epoch 00039: val_acc did not improve from 0.92377
Epoch 40/100
890/890 [==============================] - 1s 1ms/step - loss: 0.3690 - acc: 0.8966 - val_loss: 0.2943 - val_acc: 0.9372

Epoch 00040: val_acc improved from 0.92377 to 0.93722, saving model to ../data/model/pre-trained_vectors/model-epoch-40-val_acc-0.94.hdf5
Epoch 41/100
890/890 [==============================] - 1s 1ms/step - loss: 0.3362 - acc: 0.9124 - val_loss: 0.3026 - val_acc: 0.9148

Epoch 00041: val_acc did not improve from 0.93722


890/890 [==============================] - 1s 1ms/step - loss: 0.1909 - acc: 0.9494 - val_loss: 0.2278 - val_acc: 0.9596

Epoch 00080: val_acc improved from 0.95067 to 0.95964, saving model to ../data/model/pre-trained_vectors/model-epoch-80-val_acc-0.96.hdf5
Epoch 81/100
890/890 [==============================] - 1s 1ms/step - loss: 0.1899 - acc: 0.9472 - val_loss: 0.2799 - val_acc: 0.9372

Epoch 00081: val_acc did not improve from 0.95964
Epoch 82/100
890/890 [==============================] - 1s 1ms/step - loss: 0.2125 - acc: 0.9427 - val_loss: 0.2595 - val_acc: 0.9417

Epoch 00082: val_acc did not improve from 0.95964
Epoch 83/100
890/890 [==============================] - 1s 1ms/step - loss: 0.1862 - acc: 0.9539 - val_loss: 0.2709 - val_acc: 0.9372

Epoch 00083: val_acc did not improve from 0.95964
Epoch 84/100
890/890 [==============================] - 1s 1ms/step - loss: 0.2104 - acc: 0.9472 - val_loss: 0.2503 - val_acc: 0.9417

Epoch 00084: val_acc did not improve from 0.95964
